In [ ]:
import cv2
import numpy as np
import sqlite3
import csv

# Connect to SQLite database
conn = sqlite3.connect('faces.db')
cursor = conn.cursor()

# Create a table to store face data if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS faces
                  (id INTEGER PRIMARY KEY, name TEXT, usn TEXT)''')

# Load dataset from CSV with URLs
def load_dataset_from_csv(csv_file):
    dataset = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            dataset.append((row[1], row[3]))  # Name and USN
    return dataset

# Load dataset from CSV
dataset = load_dataset_from_csv('face_data.csv')

# Load the group picture
group_image_path = 'P21.png'
group_image = cv2.imread(group_image_path)

# Load cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Convert group image to grayscale
gray = cv2.cvtColor(group_image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

# Dictionary to store whether a name has already been assigned to a detected face
assigned_names = {}

# Match detected faces with dataset based on positions
for (x, y, w, h) in faces:
    matched = False
    for name, usn in dataset:
        # Check if the name has already been assigned to another detected face
        if name in assigned_names.values():
            continue  # Skip this name if it's already assigned to a face
        # Here you would typically calculate some similarity metric between the detected face and the faces in the dataset
        # For simplicity, we'll just check if the detected face is within a certain distance of the face in the dataset
        # You may need to adjust the threshold depending on your data
        if name is not dataset:
            matched = True
            # Insert face data into the database
            cursor.execute("INSERT INTO faces (name, usn) VALUES (?, ?)",
                           (name, usn))
            conn.commit()  # Commit changes

            # Draw rectangle around detected face in the group image
            cv2.rectangle(group_image, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Draw name and USN above the detected face
            cv2.putText(group_image, f"Name: {name}, USN: {usn}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            # Mark the name as assigned to this detected face
            assigned_names[(x, y, w, h)] = name
            break  # Break loop once a match is found
    if not matched:
        print("No match found for the detected face.")

# Show the group image with names, USNs, and detected faces
cv2.imshow('Group Picture with Names, USNs, and Detected Faces', group_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
